# RAG (검색 증강 생성) : FAISS 사용
: https://www.promptingguide.ai/kr/techniques/rag

문서를 벡터로 변환(임베딩) : 텍스트 데이터를 고차원 벡터 공간으로 매핑하는 작업

### FAISS (Facebook AI Similarity Search)
Facebook AI 팀에서 개발한 대규모 벡터 검색 라이브러리로, 빠르고 효율적인 최근접 이웃 검색(Nearest Neighbor Search, NNS) 을 수행하는 데 사용된다. 
<br>
특히, 고차원 벡터 검색을 최적화하여 딥러닝 임베딩 검색, 추천 시스템, RAG(Retrieval-Augmented Generation) 등에서 널리 활용된다

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

In [2]:
# ! pip install faiss-cpu
import faiss
import numpy as np

# 샘플 문서데이터 (텍스트)
documents = [
    "OpenAI는 인공지능 연구소입니다.",
    "FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.",
    "GPT는 자연어 처리를 위한 강력한 AI 모델입니다.",
    "RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다."    
]

In [3]:
# 문서를 벡터로 변환 (임베딩)
# https://platform.openai.com/docs/pricing
def get_embedding(text):
    response = client.embeddings.create(
        model = 'text-embedding-3-small',   # text-embedding-ada-002, text-embedding-3-large
        input = text        
    )
    return np.array(response.data[0].embedding)

# 모든 문서의 임베딩 생성
document_embeddings = np.array([get_embedding(doc) for doc in documents ])
print(document_embeddings)

[[-0.00736527 -0.00445131  0.0193053  ...  0.00990422 -0.00136497
   0.00519902]
 [-0.01358794  0.00780865 -0.02596267 ...  0.02078999 -0.03368309
  -0.01462468]
 [-0.01112392  0.07651846  0.03827878 ... -0.02164178 -0.00357275
   0.00447449]
 [ 0.01525772  0.02207769  0.00052901 ... -0.01211735 -0.01122917
  -0.03973562]]


In [4]:
print(document_embeddings[0])        # "OpenAI는 인공지능 연구소입니다."
print(len(document_embeddings[0]))   # 1536

[-0.00736527 -0.00445131  0.0193053  ...  0.00990422 -0.00136497
  0.00519902]
1536


In [5]:
# FAISS 벡터 인덱스 생성
dimension = document_embeddings.shape[1]   
print(dimension)    # 1536
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

1536


In [6]:
#  사용자 질문 입력 및 벡터 변환
query = "RAG의 핵심 구성 요소는 무엇인가?"
query_embedding = get_embedding(query)
print(query_embedding)

[ 0.01566273  0.00232661 -0.03758604 ... -0.0011633  -0.01976138
 -0.01448042]


In [7]:
#  FAISS를 이용한 검색( 가장 유사한 문서 찾기)
k = 2  # 가장 유사한 2개의 문서 검색
distances,indices = index.search(query_embedding.reshape(1,-1),k)
print(distances, indices)

[[1.0512233 1.575014 ]] [[3 1]]


In [8]:
# # 배열의 reshape
# a = np.arange(12)
# print(a.shape)  # 1차원 배열
# print(a)
# a2 = a.reshape(3,4)
# print(a2.shape)  # 2차원 배열
# print(a2)
# a3 = a.reshape(2,3,2) 
# print(a3.shape)  # 3차원 배열
# print(a3)

# b = a.reshape(3,-1)  # -1은 shape을 자동으로 계산
# print(b,b.shape)

In [10]:
# 검색된 문서 내용 추출
retrieved_docs = [documents[i] for i in indices[0]]
print(retrieved_docs)

['RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다.', 'FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.']


In [12]:
# LLM을 활용하여 최종 응답 생성
context = "\n".join(retrieved_docs)
prompt = f"질문: {query}\n\n참고자료:\n{context}\n\n답변:"
print(prompt)

completion = client.chat.completions.create(
    model = "gpt-4o-mini", 
    messages = [{"role": "system", "content": "당신은 유용한 AI 어시스턴트입니다."},
                {"role":"user","content":prompt} ]
)    

final_answer = completion.choices[0].message.content
print("🔹 AI 응답:", final_answer)

질문: RAG의 핵심 구성 요소는 무엇인가?

참고자료:
RAG는 검색 기반 생성 기법을 활용하여 AI 응답을 향상시키는 방법입니다.
FAISS는 Facebook AI에서 개발한 벡터 검색 라이브러리입니다.

답변:
🔹 AI 응답: RAG( retrieval-augmented generation)의 핵심 구성 요소는 크게 두 가지로 나눌 수 있습니다:

1. **검색기(Search Component)**: 이 구성 요소는 사용자의 질의에 대해 적절한 정보를 검색하기 위해 데이터베이스나 문서 집합에서 관련 데이터를 찾아내는 역할을 합니다. 일반적으로 FAISS와 같은 벡터 검색 라이브러리를 사용하여 효율적으로 유사한 문서를 찾습니다. 검색기는 사용자의 입력을 기반으로 다수의 문서를 선택하고 이를 생성 과정에서 참고할 수 있게 합니다.

2. **생성기(Generation Component)**: 이 구성 요소는 검색된 정보를 바탕으로 최종적인 텍스트 응답을 생성하는 역할을 합니다. 일반적으로 대규모 언어 모델이 사용되어, 검색된 정보를 적절히 통합하여 자연어로서 유창하고 유의미한 응답을 형성합니다.

이 두 구성 요소의 결합을 통해 RAG는 보다 정확하고 정보가 풍부한 응답을 생성할 수 있게 됩니다.
